<a href="https://colab.research.google.com/github/Jgallear/CSSP_brazil_23_24/blob/main/cross_validate_GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# cross validate the optimum number of GBM model estimators
!pip install cartopy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 1.7 MB/s eta 0:00:00


In [2]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
from sklearn.inspection import PartialDependenceDisplay
from shapely.geometry import Point
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
#from rasterio import CRS
from shapely.geometry import mapping
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
import functools
import pickle
import cartopy.feature as cf
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
import itertools
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold

In [3]:
# to determine split, take random set of 5 years, create evaluation period, then split training in half, and test number of estimators on each half,
# this will mean model is trained number of folds * number of estimators times. (2*6) times

In [4]:
# do cv split based on year, split years into 2 folds and evaluate a range of n_estimators from 2, 10, 50, 100, 200, 500

In [5]:
# load df
df = pd.read_csv('/content/drive/MyDrive/VHI_spei_rzsm_dataset.csv')
print(df)

           lon     lat      vhi-1   precip-1      rzsm-1  spei1-1m  spei2-1m  \
0      -53.375 -32.125  70.414172  259.20056   98.021910  2.192168  1.313241   
1      -53.375 -32.125  79.323750  146.58607   98.162980  1.228278  2.453098   
2      -53.375 -32.125  77.111895  184.82845   97.594270  1.472914  1.588110   
3      -53.375 -32.125  64.150561  108.86273   97.605330  1.680747  1.751371   
4      -53.375 -32.125  54.270359  145.90760  100.000000  1.641170  1.900034   
...        ...     ...        ...        ...         ...       ...       ...   
577682 -47.625  -0.875  41.102345  170.38158   66.929490  0.324701  0.572972   
577683 -47.625  -0.875  49.922513  294.35620   80.138200  1.130223  0.881931   
577684 -47.625  -0.875  52.101102  125.48111   88.888840  2.664571  2.250521   
577685 -47.625  -0.875  46.753572   24.13338   77.834076  2.523025  3.002636   
577686 -47.625  -0.875  48.627606   71.58264   69.860950  1.286047  2.275886   

        spei3-1m   spi01-1   spi02-1   

In [6]:
# need to loop through df separate specified years into train and test
n_estimators_list = [2,10,25,50,100,200,500]
# obtain random set of years for testing
print(df['year'].unique())
test_yrs = np.random.choice(df['year'].unique(),replace=False,size=5)
print(test_yrs)

[2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018 2019 2020 2021]
[2011 2004 2012 2006 2017]


In [7]:
test_yrs = [2016, 2011, 2019, 2015, 2006]

In [8]:
# remove test_yrs from dataset, then split training into fold1 and fold2
test_df = df[df['year'].isin(test_yrs)]
###
train_df = df[~df['year'].isin(test_yrs)]
# split train_df in half for fold 1 and fold 2
print(len(train_df['year'].unique()))
print(int(len(train_df['year'].unique())/2))
split = int(len(train_df['year'].unique())/2)
print(train_df['year'].unique())
f1_years = train_df['year'].unique()[:split]
print(f1_years)
f2_years = train_df['year'].unique()[split:]
print(f2_years)
## use f1 years and f2 years to split dataframe
train_df_ri = train_df.reset_index()
f1_df = train_df_ri[~train_df_ri['year'].isin(f1_years)]
f2_df = train_df_ri[~train_df_ri['year'].isin(f2_years)]

f1_inds = f1_df.index
f2_inds = f2_df.index

14
7
[2003 2004 2005 2007 2008 2009 2010 2012 2013 2014 2017 2018 2020 2021]
[2003 2004 2005 2007 2008 2009 2010]
[2012 2013 2014 2017 2018 2020 2021]


In [9]:
print(max(f1_inds))
print(max(f2_inds))

425006
424929


In [10]:
folds_list=[]
folds_list.append([f1_inds,f2_inds])
folds_list.append([f2_inds,f1_inds])

In [11]:
print(len(f1_inds))
print(len(f2_inds))

213752
211255


In [12]:
group = df.year.tolist()
target='VHI'
y = df[target]
x = df.drop(columns=[target])

In [13]:
gkf = GroupKFold(n_splits=2)
for train_index, test_index in gkf.split(x,groups=group):
  print('train: ',train_index, 'test: ', test_index)
  xtrain = x.iloc[train_index,:]
  ytrain = y[train_index]
  xtest = x.iloc[test_index,:]
  ytest = y[test_index]

train:  [    21     22     23 ... 577662 577663 577664] test:  [     0      1      2 ... 577684 577685 577686]
train:  [     0      1      2 ... 577684 577685 577686] test:  [    21     22     23 ... 577662 577663 577664]


In [14]:
print(xtrain['year'].unique())
print(xtest['year'].unique())

[2003 2004 2006 2008 2010 2014 2016 2018 2020 2021]
[2005 2007 2009 2011 2012 2013 2015 2017 2019]


In [15]:
# use grid search cv
#define estimator
GBM_model = GradientBoostingRegressor()
# optimize number of estimators
param_grid = {'n_estimators':[2,10,25,50,100,200,500]}
grid_cv = GridSearchCV(GBM_model,param_grid,cv=folds_list,scoring='r2',verbose=3)

In [16]:
print(train_df)

           lon     lat      vhi-1   precip-1      rzsm-1  spei1-1m  spei2-1m  \
0      -53.375 -32.125  70.414172  259.20056   98.021910  2.192168  1.313241   
1      -53.375 -32.125  79.323750  146.58607   98.162980  1.228278  2.453098   
2      -53.375 -32.125  77.111895  184.82845   97.594270  1.472914  1.588110   
3      -53.375 -32.125  64.150561  108.86273   97.605330  1.680747  1.751371   
4      -53.375 -32.125  54.270359  145.90760  100.000000  1.641170  1.900034   
...        ...     ...        ...        ...         ...       ...       ...   
577682 -47.625  -0.875  41.102345  170.38158   66.929490  0.324701  0.572972   
577683 -47.625  -0.875  49.922513  294.35620   80.138200  1.130223  0.881931   
577684 -47.625  -0.875  52.101102  125.48111   88.888840  2.664571  2.250521   
577685 -47.625  -0.875  46.753572   24.13338   77.834076  2.523025  3.002636   
577686 -47.625  -0.875  48.627606   71.58264   69.860950  1.286047  2.275886   

        spei3-1m   spi01-1   spi02-1   

In [17]:
trainx = train_df.drop(columns=['VHI'])
trainy = train_df['VHI']
# scale inputs
scaler = StandardScaler()
trainx_sc = scaler.fit_transform(trainx)

In [18]:
print(len(trainy))
print(len(trainx_sc))

425007
425007


In [19]:
print(trainy)

0         79.323750
1         77.111895
2         64.150561
3         54.270359
4         59.200781
            ...    
577682    49.922513
577683    52.101102
577684    46.753572
577685    48.627606
577686    47.450658
Name: VHI, Length: 425007, dtype: float64


In [20]:
grid_result = grid_cv.fit(trainx_sc,trainy)

Fitting 2 folds for each of 7 candidates, totalling 14 fits
[CV 1/2] END ....................n_estimators=2;, score=0.164 total time=   3.3s
[CV 2/2] END ....................n_estimators=2;, score=0.166 total time=   3.1s
[CV 1/2] END ...................n_estimators=10;, score=0.484 total time=  15.3s
[CV 2/2] END ...................n_estimators=10;, score=0.478 total time=  15.3s
[CV 1/2] END ...................n_estimators=25;, score=0.600 total time=  37.7s
[CV 2/2] END ...................n_estimators=25;, score=0.578 total time=  36.4s
[CV 1/2] END ...................n_estimators=50;, score=0.635 total time= 1.2min
[CV 2/2] END ...................n_estimators=50;, score=0.600 total time= 1.2min
[CV 1/2] END ..................n_estimators=100;, score=0.646 total time= 2.4min
[CV 2/2] END ..................n_estimators=100;, score=0.616 total time= 2.4min
[CV 1/2] END ..................n_estimators=200;, score=0.646 total time= 5.0min
[CV 2/2] END ..................n_estimators=200;,

In [21]:
# produce plot of model fit average r2 scores against number of estimators
print(grid_result.best_score_)
print(grid_result.best_params_)

0.6353729280663718
{'n_estimators': 200}


In [29]:
# test model with best params on hold out test dataset
GBM_model = GradientBoostingRegressor(n_estimators=200)
GBM_model.fit(trainx_sc,trainy)

GradientBoostingRegressor(n_estimators=200)

In [30]:
testx = test_df.drop(columns=['VHI'])
testy = test_df['VHI']
testx_sc = scaler.transform(testx)

In [31]:
predictions = GBM_model.predict(testx_sc)

In [32]:
print(r2_score(testy,predictions))
test_r2_score = r2_score(testy,predictions)

0.6181023272257449


In [33]:
# Get line plot of estimators vs R2 performance
# save model outputs to use in plotting script
print(len(predictions))
print(predictions.shape)
print(testx)

152680
(152680,)
           lon     lat      vhi-1    precip-1     rzsm-1  spei1-1m  spei2-1m  \
32     -53.375 -32.125  33.460501   31.077044  43.486305 -1.947763 -0.260165   
33     -53.375 -32.125  53.849729  115.865265  20.373812  0.641211 -0.586411   
34     -53.375 -32.125  62.237937   62.560310  22.396515 -1.185511 -0.421650   
35     -53.375 -32.125  60.625543  104.338350  22.017498 -0.140137 -0.993312   
36     -53.375 -32.125  64.313523   67.846820  23.099638 -0.963393 -0.731726   
...        ...     ...        ...         ...        ...       ...       ...   
577660 -47.625  -0.875  40.887218  273.721040  69.872240  0.256454 -0.125266   
577661 -47.625  -0.875  40.887218  138.636630  49.382520 -0.555288 -0.168773   
577662 -47.625  -0.875  50.892596  132.648000  52.529457  0.057421 -0.337412   
577663 -47.625  -0.875  43.292766   18.556600  48.998238  0.275302  0.111811   
577664 -47.625  -0.875  46.295286   36.166164  45.469177  1.248383  0.872078   

        spei3-1m   spi

In [34]:
# bring lat, lon, month and year together
results_df = pd.DataFrame({'lat':testx['lat'],'lon':testx['lon'],'month':testx['month'],'year':testx['year'],'predictions':predictions,'testy':testy})

In [35]:
results_df.to_csv('/content/drive/MyDrive/model_outs/GBM_out_df_optimized.csv',index=False)